In [34]:
# Análise e manipulação de dados

import numpy as np
import pandas as pd
import numpy as np
# Dados séries temporais

from datetime import datetime, timedelta
import pytz
from time import sleep

# Bibliotecas gráficas

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Integração com MetaTrader 5
import datetime as dt
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")

import pandasql as ps

In [35]:
mt5.initialize()

True

In [36]:
def baixar_dados(ticker):
    cotacoes_mt5_all = []

    timezone = pytz.timezone("America/Sao_Paulo")
    data_inicio = datetime.now() - timedelta(days=365)
    data_fim = datetime.now()

    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_H1, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)
    return pd.concat(cotacoes_mt5_all, axis=0)


In [37]:
def tratamento_df(df_final):
    df_final = df_final.drop('time', axis =1)

    df_final = df_final.reset_index()

    # Criar a coluna "data" contendo apenas a parte da data
    df_final['data'] = df_final['time'].dt.date

    # Criar a coluna "hora" contendo apenas a parte da hora
    df_final['hora'] = df_final['time'].dt.time

    df_final = df_final.drop('time', axis =1)
    return df_final

In [38]:
def window_func(df_final):
    # Sort the DataFrame by 'data' and 'ticker', then by 'hora'
    df_final.sort_values(by=['data', 'ticker', 'hora'], inplace=True)

    # Apply the window function and create a new column 'numeration'
    df_final['numeration'] = df_final.groupby(['data', 'ticker']).cumcount() + 1
        
    # Find the maximum 'numeration' for each day and ticker
    max_numeration_per_day = df_final.groupby(['data', 'ticker'])['numeration'].max()

    # Merge the result back into the original DataFrame to get the closing value (close) for each day
    df_final = df_final.merge(max_numeration_per_day, on=['data', 'ticker'], suffixes=('', '_max'))

    closing_values_per_day = df_final[df_final['numeration'] == df_final['numeration_max']][['data', 'ticker', 'close']]

    df_final = df_final.merge(closing_values_per_day, on=['data', 'ticker'], how='left')

    df_final.rename(columns={'close_y': 'fechamento_do_dia'}, inplace=True)
    #df_final.drop(['numeration','numeration_max'], inplace=True, axis=1)
    return df_final

In [39]:
def dia_anterior(df):
    df_anterior = df[['data', 'fechamento_do_dia']].groupby('data').last().reset_index()
    df_anterior.columns = ['data', 'fechamento_dia_anterior']

    df_anterior['fechamento_dia_anterior'] = df_anterior['fechamento_dia_anterior'].shift(1)

    # Faça um left join com base na coluna 'data'
    df_resultado = pd.merge(df, df_anterior, on='data', how='left')
    return df_resultado

In [40]:
def create_perc(df_ativo, percentual):
    #Ajustar variáveis
    df_ativo['entrada'] = df_ativo['fechamento_dia_anterior'] - (df_ativo['fechamento_dia_anterior'] * percentual)
    df_ativo['percentual'] = percentual
    return df_ativo

In [41]:
def create_stats (df_ativo, dias_analisados, horario):
    df_ativo['numeration'] = df_ativo['numeration'].astype(int)
    df_ativo = df_ativo.loc[df_ativo['numeration'] == horario]

    df_ativo['tem_entrada'] = np.where(df_ativo['open'] <= df_ativo['entrada'],1,0)
    df_ativo['alvo'] = np.where(df_ativo['tem_entrada'] == 1 , (df_ativo['close_x'] / df_ativo['open'] -1) *100, 0) 

    df_filtrado = df_ativo.loc[df_ativo['tem_entrada']==1]
    df_filtrado = df_filtrado.tail(dias_analisados)
    df_filtrado['alvo_acumulado'] = df_filtrado['alvo'].cumsum()
    return df_filtrado

In [42]:
def calcular_drawdown(df_filtrado, horario):
    df_filtrado['max_acumulado'] = df_filtrado['alvo_acumulado'].cummax()
    df_filtrado['drawdown'] = df_filtrado['alvo_acumulado'] - df_filtrado['max_acumulado']

    # Calcular o drawdown máximo
    drawdown_maximo = df_filtrado['drawdown'].min()

    # Localizar o índice do valor mínimo em 'drawdown'
    indice_drawdown_maximo = df_filtrado['drawdown'].idxmin()

    # Acessar a data correspondente ao índice do drawdown máximo
    data_drawdown_maximo = df_filtrado.loc[indice_drawdown_maximo, 'data']

    # Calcular a taxa de acerto
    resultado = df_filtrado['alvo'] > 0
    taxa_acerto = sum(resultado) / len(df_filtrado['alvo'])

    # Separar resultados positivos e negativos
    resultados_positivos = df_filtrado.loc[df_filtrado['alvo'] > 0, 'alvo']
    resultados_negativos = df_filtrado.loc[df_filtrado['alvo'] <= 0, 'alvo']

    try:
        pior_dia = min(resultados_negativos)
        melhor_dia = max(resultados_positivos)
    except:
        pior_dia = 0
        melhor_dia = 0
        
    indice_pior_dia = df_filtrado['alvo'].idxmin()
    data_pior_dia = df_filtrado.loc[indice_pior_dia, 'data']
    
    quantidade_trades = len(resultados_positivos) + len(resultados_negativos)
    # Calcular médias
    media_positivos = resultados_positivos.mean()
    media_negativos = resultados_negativos.mean()
    media_geral = df_filtrado['alvo'].mean()

    # Calcular frequência de resultados positivos e negativos
    freq_positivos = len(resultados_positivos) / len(df_filtrado)
    freq_negativos = len(resultados_negativos) / len(df_filtrado)

    # Calcular expectativa matemática
    expectativa = freq_positivos * media_positivos - freq_negativos * abs(media_negativos)

    # Calcular retorno médio diário
    retorno_medio_diario = df_filtrado['alvo'].mean()

    # Calcular desvio padrão diário
    desvio_padrao_diario = df_filtrado['alvo'].std()

    # Calcular índice de Sharpe
    indice_sharpe = np.sqrt(252) * retorno_medio_diario / desvio_padrao_diario
    
    ativo =  df_filtrado.iloc[2]['ticker']
    retorno_acumulado = df_filtrado.iloc[-1]['alvo_acumulado']
    primeiro_dia = df_filtrado.iloc[1]['data']
    
    resultado_dict = {
        'ativo': ativo,
        'horário': horario,
        'Retorno Acumulado': retorno_acumulado,
        'Melhor dia': melhor_dia,           
        'Pior Dia':pior_dia, 
        'Data Pior dia': data_pior_dia,
        'Primeiro Dia': primeiro_dia,
        'Quantidade de Trades': quantidade_trades,
        'Taxa de Acerto': taxa_acerto,
        'Media Geral': media_geral,
        'Média Positivos': media_positivos,
        'Média Negativos': media_negativos,
        'Expectativa Matemática': expectativa,
        'Índice de Sharpe': indice_sharpe,
        'Drawdown Máximo': drawdown_maximo,
        'Data do Drawdown': data_drawdown_maximo
    }

    return resultado_dict

In [43]:
ibov = ['SEQL3',
'AERI3',
'WEST3',
'TRAD3',
'KRSA3',
'ESPA3',
'IFCM3',
'NGRD3',
'VIIA3',
'MEAL3',
'SGPS3',
'BMGB4',
'LPSB3',
'PDTC3',
'SHOW3',
'CVCB3',
'LUPA3',
'MGLU3',
'UCAS3',
'BRIT3',
'MLAS3',
'COGN3',
'TECN3',
'PINE4',
'NINJ3',
'TCSA3',
'MBLY3',
'HBSA3',
'RAIZ4',
'CIEL3',
'HBOR3',
'CMIN3',
'SYNE3',
'PGMN3',
'QUAL3',
'PFRM3',
'ZAMP3',
'FIQE3',
'OSXB3',
'CSED3',
'POMO3',
'ANIM3',
'HAPV3',
'DMVF3',
'MELK3',
'AGXY3',
'ALPK3',
'CBAV3',
'CLSA3',
'OPCT3',
'PDGR3',
'LVTC3',
'JHSF3',
'RDNI3',
'POMO4',
'HBRE3',
'GFSA3',
'CEAB3',
'SEER3',
'TRIS3',
'LJQQ3',
'WIZC3',
'GUAR3',
'ALLD3',
'AVLL3',
'PETZ3',
'USIM5',
'LWSA3',
'USIM3',
'GRND3',
'MTRE3',
'CAML3',
'GMAT3',
'BOAS3',
'EVEN3',
'MRFG3',
'JALL3',
'PTBL3',
'ECOR3',
'CPLE6',
'BPAN4',
'POSI3',
'DEXP4',
'LAVV3',
'ALPA4',
'DXCO3',
'GOLL4',
'CASH3',
'DEXP3',
'ITSA4',
'ITSA3',
'STBP3',
'RAPT3',
'KEPL3',
'ALPA3',
'BRFS3',
'JSLG3',
'MATD3',
'BEEF3',
'CXSE3',
'RANI3',
'SIMH3',
'PLPL3',
'SOMA3',
'AESB3',
'MEGA3',
'MDNE3',
'ONCO3',
'BMEB4',
'SOJA3',
'CRFB3',
'VITT3',
'VAMO3',
'PRNR3',
'CEDO4',
'MOVI3',
'CMIG4',
'ODPV3',
'RAPT4',
'TTEN3',
'MILS3',
'FHER3',
'ENEV3',
'CCRO3',
'PNVL3',
'SBFG3',
'GOAU3',
'EUCA4',
'BMEB3',
'PORT3',
'TEND3',
'GOAU4',
'FRAS3',
'DASA3',
'ASAI3',
'AURE3',
'MRVE3',
'BMOB3',
'BBDC3',
'TFCO4',
'BRSR3',
'ARML3',
'ENAT3',
'ROMI3',
'BRSR6',
'DESK3',
'TIMS3',
'MYPK3',
'B3SA3',
'ELMD3',
'CSUD3',
'BBDC4',
'TASA4',
'TASA3',
'FLRY3',
'BRBI11',
'VLID3',
'VBBR3',
'AZUL4',
'EUCA3',
'CURY3',
'EMBR3',
'NTCO3',
'CSMG3',
'GGPS3',
'CMIG3',
'UGPA3',
'NEOE3',
'ABCB4',
'LREN3',
'CSAN3',
'JBSS3',
'PCAR3',
'LOGG3',
'VVEO3',
'VULC3',
'SAPR11',
'BLAU3',
'YDUQ3',
'BRAP3',
'DIRR3',
'IGTI11',
'AMBP3',
'RECV3',
'EZTC3',
'RAIL3',
'BRAP4',
'KLBN11',
'BRKM5',
'INTB3',
'SCAR3',
'SMFT3',
'ALSO3',
'TGMA3',
'AALR3',
'ITUB3',
'BRKM3',
'OFSA3',
'CYRE3',
'LAND3',
'TRPL4',
'GGBR3',
'AGRO3',
'SQIA3',
'MULT3',
'PSSA3',
'RADL3',
'ITUB4',
'GGBR4',
'TUPY3',
'ALUP11',
'TOTS3',
'PETR4',
'VIVA3',
'APER3',
'BBSE3',
'EQTL3',
'PETR3',
'BPAC11',
'SMTO3',
'TRPL3',
'RDOR3',
'CPFE3',
'ELET3',
'TAEE11',
'RRRP3',
'ORVR3',
'IRBR3',
'WEGE3',
'ELET6',
'MDIA3',
'EGIE3',
'HYPE3',
'SLCE3',
'LEVE3',
'BBAS3',
'PRIO3',
'LOGN3',
'ENGI11',
'SUZB3',
'FESA4',
'SBSP3',
'CLSC3',
'CLSC4',
'RENT3',
'VALE3',
'ARZZ3']

In [44]:
horarios = [2,3,4,5,6,7,8,9]
lista_resultados = []
for ativo in ibov:
    print( f'Baixando os dados de {ativo}')
    for hora in horarios:
        try:
            df_final = baixar_dados(ativo)
            df_final = tratamento_df(df_final)
            df_final = window_func(df_final)
            df_final = dia_anterior(df_final)
            df_final = create_perc(df_final, 0.01)
            df_final = create_stats(df_final, dias_analisados = 20,horario = hora)
            resultado = calcular_drawdown(df_final, hora)
            lista_resultados.append(resultado)
        except:
            continue

df_resultado = pd.DataFrame(lista_resultados)

Baixando os dados de SEQL3
Baixando os dados de AERI3
Baixando os dados de WEST3
Baixando os dados de TRAD3
Baixando os dados de KRSA3
Baixando os dados de ESPA3
Baixando os dados de IFCM3
Baixando os dados de NGRD3
Baixando os dados de VIIA3
Baixando os dados de MEAL3
Baixando os dados de SGPS3
Baixando os dados de BMGB4
Baixando os dados de LPSB3
Baixando os dados de PDTC3
Baixando os dados de SHOW3
Baixando os dados de CVCB3
Baixando os dados de LUPA3
Baixando os dados de MGLU3
Baixando os dados de UCAS3
Baixando os dados de BRIT3
Baixando os dados de MLAS3
Baixando os dados de COGN3
Baixando os dados de TECN3
Baixando os dados de PINE4
Baixando os dados de NINJ3
Baixando os dados de TCSA3
Baixando os dados de MBLY3
Baixando os dados de HBSA3
Baixando os dados de RAIZ4
Baixando os dados de CIEL3
Baixando os dados de HBOR3
Baixando os dados de CMIN3
Baixando os dados de SYNE3
Baixando os dados de PGMN3
Baixando os dados de QUAL3
Baixando os dados de PFRM3
Baixando os dados de ZAMP3
B

In [45]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] >= 0.75) 
                    & (df_resultado['Pior Dia'] > -1.5) 
                    & (df_resultado['Media Geral'] > 0.3)
                    & (df_resultado['Quantidade de Trades'] >= 20)
                    ].sort_values(['Pior Dia'], ascending=False)
selecionados

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
1287,BRAP4,8,7.441684,1.003344,-0.133023,2023-01-23,2022-12-12,20,0.85,0.372084,0.450113,-0.070081,0.372084,17.389600,-0.133023,2023-01-23
1503,SMTO3,7,7.257845,1.373626,-0.436554,2023-11-09,2023-08-22,20,0.75,0.362892,0.582186,-0.294988,0.362892,10.686586,-1.049636,2023-11-10
1476,EQTL3,2,7.322209,1.142298,-0.590624,2023-04-25,2023-03-22,20,0.75,0.366110,0.597183,-0.327108,0.366110,10.899211,-0.627799,2023-04-26
928,AURE3,8,6.854380,1.274900,-0.617761,2023-01-03,2022-12-20,20,0.75,0.342719,0.604141,-0.441546,0.342719,10.249054,-1.381143,2023-01-09
1465,APER3,5,11.025889,2.022245,-0.662252,2023-08-14,2023-07-12,20,0.80,0.551294,0.747179,-0.232244,0.551294,12.549996,-0.762586,2023-08-14
1657,SBSP3,8,7.930956,2.173509,-0.707408,2023-02-28,2022-12-06,20,0.80,0.396548,0.610114,-0.457718,0.396548,8.606459,-1.137768,2023-03-09
1363,CYRE3,8,6.671141,2.608099,-0.918197,2023-01-03,2022-12-26,20,0.75,0.333557,0.579256,-0.403541,0.333557,7.369630,-0.969724,2023-02-24
1630,ENGI11,2,7.410869,1.539168,-0.945471,2023-06-22,2023-06-01,20,0.75,0.370543,0.647626,-0.460704,0.370543,8.385588,-0.945471,2023-06-22
1245,DIRR3,8,7.151230,1.686122,-1.024259,2023-11-06,2023-01-11,20,0.75,0.357562,0.654796,-0.534143,0.357562,8.437202,-1.024259,2023-11-06
532,ECOR3,3,7.194779,2.367688,-1.052632,2023-10-13,2023-08-07,20,0.75,0.359739,0.575458,-0.287418,0.359739,8.733825,-1.052632,2023-10-13


2 - 11:00

3 - 12:00

4 - 13:00

5 - 14:00

6 - 15:00

7 - 16:00

8 - 17:00

Comprar a abertura do horário acima

In [46]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] <= 0.25) 
                    & (df_resultado['Melhor dia'] < 1.5)
                    & (df_resultado['Quantidade de Trades'] >= 20)
                    ].sort_values(['Retorno Acumulado'], ascending=False)
selecionados

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
957,TFCO4,8,0.433073,1.060606,-1.609195,2023-10-02,2023-08-15,20,0.20,0.021654,0.680347,-0.143020,0.021654,0.683003,-1.609195,2023-10-02
81,LPSB3,6,0.152331,1.500000,-1.000000,2023-10-06,2023-09-18,20,0.25,0.007617,0.865690,-0.278408,0.007617,0.185659,-1.490196,2023-10-18
958,TFCO4,9,0.096154,0.096154,0.000000,2022-12-13,2022-12-16,20,0.05,0.004808,0.096154,0.000000,0.004808,3.549648,0.000000,2022-12-13
1678,VALE3,6,-0.405297,0.458913,-0.243235,2023-10-18,2023-07-17,20,0.25,-0.020265,0.201883,-0.094314,-0.020265,-1.888813,-1.051873,2023-10-20
472,USIM3,6,-1.043419,0.737463,-0.647249,2023-10-20,2023-07-28,20,0.20,-0.052171,0.388712,-0.162392,-0.052171,-2.576168,-2.003671,2023-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,ANIM3,5,-14.318379,1.072386,-2.318841,2023-09-05,2023-08-15,20,0.15,-0.715919,0.715660,-0.968551,-0.715919,-12.488838,-13.600484,2023-10-30
356,LVTC3,8,-14.369982,0.900901,-3.899083,2023-01-30,2023-01-16,20,0.20,-0.718499,0.431490,-1.005996,-0.718499,-11.205936,-13.960705,2023-03-10
389,GFSA3,6,-14.948525,0.678733,-6.387665,2023-10-20,2023-09-05,20,0.25,-0.747426,0.385421,-1.125042,-0.747426,-7.442896,-15.158609,2023-11-09
98,CVCB3,2,-16.180584,1.286174,-6.390977,2023-08-31,2023-08-21,20,0.25,-0.809029,0.637538,-1.291218,-0.809029,-7.629049,-16.682444,2023-11-13
